Connected to .venv (Python 3.10.16)

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 20),
        "batch_size": trial.suggest_int("batch_size", 4, 16)
    }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True):
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37,
            encoder_sizes=(128,128),
            decoder_sizes=(256,256),
            kernel="cauchy",
            beta=0.2,
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=7.0, #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)

        mae_list, rmse_list = [], []
        for i in range(n_features):
            mask_i = test_mask[:, :, i]
            pred = test_imputation_denorm[:, :, i][mask_i]
            true = test_ori_denorm[:, :, i][mask_i]
            if len(true) == 0: continue
            mae = np.mean(np.abs(pred - true))
            rmse = np.sqrt(mean_squared_error(true, pred))
            mae_list.append(mae)
            rmse_list.append(rmse)
            mlflow.log_metric(f"MAE_{i}", mae)
            mlflow.log_metric(f"RMSE_{i}", rmse)

        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        return np.mean(mae_list), np.mean(rmse_list)

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study") as parent_run:
    study = optuna.create_study(directions=["minimize","minimize"])
    study.optimize(objective, n_trials=10)

    best_params = study.best_trial.params
    best_value = study.best_trial.value # or best_trial.values if multi-objective

    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

CUDA available: True
Using CUDA


2025/05/19 22:50:13 INFO mlflow.tracking.fluent: Experiment with name 'GP_VAE_2' does not exist. Creating a new experiment.
2025/05/19 22:50:13 INFO mlflow.tracking.fluent: Experiment with name 'GP-VAE-2' does not exist. Creating a new experiment.
[I 2025-05-19 22:50:13,818] A new study created in memory with name: no-name-80320a2b-4c2d-4a98-b708-5ed8770305df
2025-05-19 22:50:13 [INFO]: Using the given device: cuda
2025-05-19 22:50:13 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250519_T225013
2025-05-19 22:50:13 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250519_T225013/tensorboard
2025-05-19 22:50:13 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678
2025-05-19 22:50:16 [INFO]: Epoch 001 - training loss (default): 11669.4683, validatio

...

2025-05-19 22:57:17 [INFO]: Epoch 001 - training loss (default): 10024.7653, validation loss: 21260.5430
2025-05-19 22:57:21 [INFO]: Epoch 002 - training loss (default): 9128.0963, validation loss: 20834.3593
2025-05-19 22:57:24 [INFO]: Epoch 003 - training loss (default): 9126.5857, validation loss: 20615.0654
2025-05-19 22:57:27 [INFO]: Epoch 004 - training loss (default): 9125.7249, validation loss: 20464.5100
2025-05-19 22:57:30 [INFO]: Epoch 005 - training loss (default): 9125.1479, validation loss: 20347.4020
2025-05-19 22:57:33 [INFO]: Epoch 006 - training loss (default): 9124.7165, validation loss: 20242.6674
2025-05-19 22:57:36 [INFO]: Epoch 007 - training loss (default): 9124.4796, validation loss: 20153.9985
2025-05-19 22:57:41 [INFO]: Epoch 008 - training loss (default): 9124.2670, validation loss: 20077.9207
2025-05-19 22:57:45 [INFO]: Epoch 009 - training loss (default): 9124.0293, validation loss: 19998.7656
2025-05-19 22:57:49 [INFO]: Epoch 010 - training loss (default)

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[I 2025-05-19 22:58:03,570] Trial 9 finished with values: [331623.7726320681, 474579.2345077271] and parameters: {'lr': 0.0003217054036792562, 'epochs': 14, 'batch_size': 9}.


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/7060a3a2bca74d1cac7da85e22beed7b
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study at: http://localhost:5000/#/experiments/832352739106302318/runs/2b73088bf5864b02ba1e8eef1c6d61e3
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


RuntimeError: A single best trial cannot be retrieved from a multi-objective study. Consider using Study.best_trials to retrieve a list containing the best trials.

In [ ]:
feature_names

['fr_eng',
 'te_exh_cyl_out__0',
 'pd_air_ic__0',
 'pr_exh_turb_out__0',
 'te_air_ic_out__0',
 'te_seawater',
 'te_air_comp_in_a__0',
 'te_air_comp_in_b__0',
 'fr_tc__0',
 'pr_baro',
 'pd_air_ic__0_1',
 'pr_exh_rec',
 'te_exh_turb_in__0',
 'te_exh_turb_out__0',
 'bo_aux_blower_running',
 're_eng_load',
 'pr_air_scav_ecs',
 'pr_air_scav',
 'te_air_scav_rec',
 'te_air_ic_out__0_1',
 'pr_cyl_comp__0',
 'pr_cyl_max__0',
 'se_mip__0',
 'te_exh_cyl_out__0_1',
 'fr_eng_setpoint',
 'te_air_scav_rec_iso',
 'pr_cyl_max_mv_iso',
 'pr_cyl_comp_mv_iso',
 'fr_eng_ecs',
 'pr_air_scav_iso',
 'engine_type_G80ME-C9.5-GI-LPSCR']

In [ ]:
df1["te_exh_cyl_out__0"].std()

39.82180184402368

In [ ]:
df1["fr_eng"].std()

0.14288534662599145

In [ ]:
df1["te_air_scav_rec"].std()

2.2921270072758015

In [ ]:
df1.describe()

,time,fr_eng,te_exh_cyl_out__0,pd_air_ic__0,pr_exh_turb_out__0,te_air_ic_out__0,te_seawater,te_air_comp_in_a__0,te_air_comp_in_b__0,fr_tc__0,...,pr_cyl_max__0,se_mip__0,te_exh_cyl_out__0_1,fr_eng_setpoint,te_air_scav_rec_iso,pr_cyl_max_mv_iso,pr_cyl_comp_mv_iso,fr_eng_ecs,pr_air_scav_iso,engine_type_G80ME-C9.5-GI-LPSCR
count,105527,105527.000000,105527.000000,105527.000000,0.0,105527.000000,1.055270e+05,0.0,0.0,0.0,...,1.053790e+05,1.053790e+05,105527.000000,105527.000000,63515.000000,6.268200e+04,6.268200e+04,105527.000000,62682.000000,105527.0
mean,2024-03-04 03:38:36.214807296,0.832822,529.817308,3111.211728,NaN,307.180888,2.731500e+02,NaN,NaN,NaN,...,1.451904e+07,9.473421e+05,529.817308,0.832486,304.498497,1.456075e+07,1.077581e+07,0.832822,99827.213871,1.0
min,2023-10-01 05:00:00,0.169932,319.150000,0.000000,NaN,293.350000,2.731500e+02,NaN,NaN,NaN,...,4.044394e+06,-3.990992e+04,319.150000,0.000000,299.030721,4.031951e+06,4.143255e+06,0.169932,970.760107,1.0
25%,2023-12-12 22:18:30,0.777478,513.150000,1770.000000,NaN,305.850000,2.731500e+02,NaN,NaN,NaN,...,1.282290e+07,7.899361e+05,513.150000,0.778309,302.687664,1.300370e+07,9.346764e+06,0.777478,60163.341706,1.0
50%,2024-03-05 10:30:00,0.897176,542.150000,3290.000000,NaN,307.150000,2.731500e+02,NaN,NaN,NaN,...,1.566881e+07,1.045076e+06,542.150000,0.899969,303.826959,1.551522e+07,1.157614e+07,0.897176,106679.359693,1.0
75%,2024-05-13 19:55:30,0.917475,553.150000,4370.000000,NaN,308.150000,2.731500e+02,NaN,NaN,NaN,...,1.637885e+07,1.121633e+06,553.150000,0.916650,307.118440,1.628880e+07,1.227650e+07,0.917475,131557.217291,1.0
max,2024-07-31 16:36:00,1.022311,597.150000,10000.000000,NaN,318.750000,2.731500e+02,NaN,NaN,NaN,...,1.882522e+07,1.513627e+06,597.150000,1.016633,310.080785,1.897977e+07,1.560624e+07,1.022311,225486.996753,1.0
std,NaN,0.142885,39.821802,1808.911991,NaN,1.789162,5.684369e-14,NaN,NaN,NaN,...,2.560751e+06,2.658281e+05,39.821802,0.143879,2.474538,2.467797e+06,2.251713e+06,0.142885,52369.802338,0.0


In [ ]:
df1["pr_baro"].std()

759.7741848141841

In [ ]:
df1["pr_baro"].std()

759.7741848141841

In [ ]:
df1["pr_cyl_max__0"].std()

2560750.9406238147

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0),
        "kernel":trial.suggest_categorical("kernel",["cauchy", "diffusion", "rbf", "matern"]),



    }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True):
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel=params["kernel"],
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)

        mae_list, rmse_list = [], []
        for i in range(n_features):
            mask_i = test_mask[:, :, i]
            pred = test_imputation_denorm[:, :, i][mask_i]
            true = test_ori_denorm[:, :, i][mask_i]
            if len(true) == 0: continue
            mae = np.mean(np.abs(pred - true))
            rmse = np.sqrt(mean_squared_error(true, pred))
            mae_list.append(mae)
            rmse_list.append(rmse)
            mlflow.log_metric(f"MAE_{i}", mae)
            mlflow.log_metric(f"RMSE_{i}", rmse)

         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        #avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        trial.set_user_attr("model_state_dict", GPVAE.state_dict())
        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/15c38ce0534446fcb51213a37a58a9b7
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/a1950ed8c0a04febb7e4910ed83d7038
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


RuntimeError: Training got interrupted. Model was not trained. Please investigate the error printed above.

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0),
        "kernel":trial.suggest_categorical("kernel",["cauchy", "diffusion", "rbf"]),



    }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True):
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel=params["kernel"],
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)

        mae_list, rmse_list = [], []
        for i in range(n_features):
            mask_i = test_mask[:, :, i]
            pred = test_imputation_denorm[:, :, i][mask_i]
            true = test_ori_denorm[:, :, i][mask_i]
            if len(true) == 0: continue
            mae = np.mean(np.abs(pred - true))
            rmse = np.sqrt(mean_squared_error(true, pred))
            mae_list.append(mae)
            rmse_list.append(rmse)
            mlflow.log_metric(f"MAE_{i}", mae)
            mlflow.log_metric(f"RMSE_{i}", rmse)

         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        #avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        trial.set_user_attr("model_state_dict", GPVAE.state_dict())
        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

CUDA available: True
Using CUDA


2025-05-20 00:37:22 [INFO]: Epoch 001 - training loss (default): 29023.6647, validation loss: 36484.5052
2025-05-20 00:37:22 [INFO]: Epoch 002 - training loss (default): 12432.6056, validation loss: 35268.0334
2025-05-20 00:37:23 [INFO]: Epoch 003 - training loss (default): 10335.1963, validation loss: 34782.2183
2025-05-20 00:37:24 [INFO]: Epoch 004 - training loss (default): 10241.5189, validation loss: 34596.8290
2025-05-20 00:37:24 [INFO]: Epoch 005 - training loss (default): 10221.6641, validation loss: 34481.5977
2025-05-20 00:37:25 [INFO]: Epoch 006 - training loss (default): 10211.6875, validation loss: 34380.6163
2025-05-20 00:37:26 [INFO]: Epoch 007 - training loss (default): 10205.3616, validation loss: 34282.8368
2025-05-20 00:37:26 [INFO]: Epoch 008 - training loss (default): 10200.9662, validation loss: 34190.5156
2025-05-20 00:37:27 [INFO]: Epoch 009 - training loss (default): 10197.9365, validation loss: 34100.7474
2025-05-20 00:37:28 [INFO]: Epoch 010 - training loss (

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[W 2025-05-20 00:37:43,270] Trial 0 failed with parameters: {'lr': 0.0003174471955074421, 'epochs': 33, 'batch_size': 128, 'length_scale': 1.9734374565972177, 'beta': 0.9182668977722848, 'kernel': 'rbf'} because of the following error: AttributeError("type object 'GPVAE' has no attribute 'state_dict'").
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-13-0ad50cddbc5f>", line 363, in objective
    trial.set_user_attr("model_state_dict", GPVAE.state_dict())
AttributeError: type object 'GPVAE' has no attribute 'state_dict'
[W 2025-05-20 00:37:43,272] Trial 0 failed with value None.


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/099574dbb9394265b2d09aab8318e34b
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/63f9b9e323ea4559a36fb66b465aaa86
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


AttributeError: type object 'GPVAE' has no attribute 'state_dict'

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0),
        "kernel":trial.suggest_categorical("kernel",["cauchy", "diffusion", "rbf"]),



    }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True):
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel=params["kernel"],
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)

        mae_list, rmse_list = [], []
        for i in range(n_features):
            mask_i = test_mask[:, :, i]
            pred = test_imputation_denorm[:, :, i][mask_i]
            true = test_ori_denorm[:, :, i][mask_i]
            if len(true) == 0: continue
            mae = np.mean(np.abs(pred - true))
            rmse = np.sqrt(mean_squared_error(true, pred))
            mae_list.append(mae)
            rmse_list.append(rmse)
            mlflow.log_metric(f"MAE_{i}", mae)
            mlflow.log_metric(f"RMSE_{i}", rmse)

         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        #avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[W 2025-05-20 00:38:42,179] Trial 0 failed with parameters: {'lr': 0.00014138309898143063, 'epochs': 13, 'batch_size': 64, 'length_scale': 2.0986642679908574, 'beta': 0.5303956187801935, 'kernel': 'rbf'} because of the following error: NameError("name 'run' is not defined").
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-14-321dd7e3a26f>", line 363, in objective
    trial.set_user_attr("mlflow_run_id", run.info.run_id)
NameError: name 'run' is not defined
[W 2025-05-20 00:38:42,181] Trial 0 failed with value None.


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/0ab070eef2004f2992b99541db2825f4
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/13d977d4fd4a4eddba68fcd03986c20a
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


NameError: name 'run' is not defined

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0),
        "kernel":trial.suggest_categorical("kernel",["cauchy", "diffusion", "rbf"]),
 }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True) as run:
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel=params["kernel"],
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)

        mae_list, rmse_list = [], []
        for i in range(n_features):
            mask_i = test_mask[:, :, i]
            pred = test_imputation_denorm[:, :, i][mask_i]
            true = test_ori_denorm[:, :, i][mask_i]
            if len(true) == 0: continue
            mae = np.mean(np.abs(pred - true))
            rmse = np.sqrt(mean_squared_error(true, pred))
            mae_list.append(mae)
            rmse_list.append(rmse)
            mlflow.log_metric(f"MAE_{i}", mae)
            mlflow.log_metric(f"RMSE_{i}", rmse)

         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        #avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

2025-05-20 00:40:02 [INFO]: Using the given device: cuda
2025-05-20 00:40:02 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004002
2025-05-20 00:40:02 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004002/tensorboard
2025-05-20 00:40:02 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678
2025-05-20 00:40:03 [INFO]: Epoch 001 - training loss (default): 722.8419, validation loss: 354.8317
2025-05-20 00:40:04 [INFO]: Epoch 002 - training loss (default): 565.1318, validation loss: 343.4969
2025-05-20 00:40:04 [INFO]: Epoch 003 - training loss (default): 552.5871, validation loss: 340.6999
2025-05-20 00:40:05 [INFO]: Epoch 004 - training loss (default): 550.3879, validation loss: 339.4278
2025-05-20 00:40:06 [INFO]: Epoch 005 - trai

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[I 2025-05-20 00:40:13,677] Trial 0 finished with value: 7675.04213732518 and parameters: {'lr': 0.00048532276594338386, 'epochs': 16, 'batch_size': 128, 'length_scale': 1.6539034791803948, 'beta': 0.38234690972619034, 'kernel': 'cauchy'}. Best is trial 0 with value: 7675.04213732518.
2025-05-20 00:40:13 [INFO]: Using the given device: cuda
2025-05-20 00:40:13 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004013
2025-05-20 00:40:13 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004013/tensorboard
2025-05-20 00:40:13 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678
2025-05-20 00:40:13 [ERROR]: ❌ Exception: Expected parameter covariance_matrix (Tensor of shape (37, 20, 20)) of distribution MultivariateNormal(loc: torch.Size([

🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/f007d5e783e64b42a7bd6742a6301f95
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/2d67f3f8141842a58203908cd6ec7f57
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/c81e5b54f26545c38c34f7e47d9768cd
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


RuntimeError: Training got interrupted. Model was not trained. Please investigate the error printed above.

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0)
 }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True) as run:
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel="cauchy",
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)


         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        #avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

2025-05-20 00:43:13 [INFO]: Using the given device: cuda
2025-05-20 00:43:13 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004313
2025-05-20 00:43:13 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004313/tensorboard
2025-05-20 00:43:13 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678
2025-05-20 00:43:15 [INFO]: Epoch 001 - training loss (default): 1974.3210, validation loss: 4101.7576
2025-05-20 00:43:16 [INFO]: Epoch 002 - training loss (default): 1702.0264, validation loss: 4097.6497
2025-05-20 00:43:17 [INFO]: Epoch 003 - training loss (default): 1700.6533, validation loss: 4094.6074
2025-05-20 00:43:18 [INFO]: Epoch 004 - training loss (default): 1699.8676, validation loss: 4090.9239
2025-05-20 00:43:19 [INFO]: Epoch 00

The shape of gp_vae_imputation is (1056, 2, 20, 31)
🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/d2cd207c3fbb495ca3e178b6c618899f
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/63c95a2dd80d44c28599e6f3d7c72d94
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


NameError: name 'rmse' is not defined

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0)
 }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True) as run:
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel="cauchy",
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)


         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse_i)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        #avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", np.mean(mae_list))
        mlflow.log_metric("avg_rmse", np.mean(rmse_list))

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

2025-05-20 00:44:00 [INFO]: Using the given device: cuda
2025-05-20 00:44:00 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004400
2025-05-20 00:44:00 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004400/tensorboard
2025-05-20 00:44:00 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678
2025-05-20 00:44:01 [INFO]: Epoch 001 - training loss (default): 869.0620, validation loss: 609.1907
2025-05-20 00:44:03 [INFO]: Epoch 002 - training loss (default): 632.1673, validation loss: 598.7252
2025-05-20 00:44:04 [INFO]: Epoch 003 - training loss (default): 622.1954, validation loss: 593.6872
2025-05-20 00:44:05 [INFO]: Epoch 004 - training loss (default): 619.6878, validation loss: 590.0230
2025-05-20 00:44:06 [INFO]: Epoch 005 - trai

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[W 2025-05-20 00:44:43,686] Trial 0 failed with parameters: {'lr': 0.0001126821627210784, 'epochs': 35, 'batch_size': 32, 'length_scale': 1.7300271829834157, 'beta': 0.5425219729252396} because of the following error: NameError("name 'mae_list' is not defined").
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-17-6cab5f49d7d7>", line 344, in objective
    mlflow.log_metric("avg_mae", np.mean(mae_list))
NameError: name 'mae_list' is not defined
[W 2025-05-20 00:44:43,690] Trial 0 failed with value None.


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/dbb34cdee13246a899d1d963a34c5323
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/1c25fd28eb544ff5ab730d7c69e4c280
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


NameError: name 'mae_list' is not defined

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0)
 }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True) as run:
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel="cauchy",
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)


         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse_i)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", avg_mae)
        mlflow.log_metric("avg_rmse", avg_rmse)

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[I 2025-05-20 00:46:32,375] Trial 0 finished with value: 7681.086558596051 and parameters: {'lr': 0.00035745162270512613, 'epochs': 30, 'batch_size': 128, 'length_scale': 2.0077899412830975, 'beta': 0.7618304568241496}. Best is trial 0 with value: 7681.086558596051.
2025-05-20 00:46:32 [INFO]: Using the given device: cuda
2025-05-20 00:46:32 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004632
2025-05-20 00:46:32 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T004632/tensorboard
2025-05-20 00:46:32 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/a0bb5ff8807f4059a8d3e3bff6fd3cb4
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


...

🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/894629cbddc54ec785154a8a0662012a
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


2025-05-20 00:50:04 [INFO]: Epoch 001 - training loss (default): 502.2495, validation loss: 123.2171
2025-05-20 00:50:06 [INFO]: Epoch 002 - training loss (default): 473.7430, validation loss: 121.9633
2025-05-20 00:50:07 [INFO]: Epoch 003 - training loss (default): 472.7724, validation loss: 120.4752
2025-05-20 00:50:08 [INFO]: Epoch 004 - training loss (default): 471.6438, validation loss: 118.5636
2025-05-20 00:50:10 [INFO]: Epoch 005 - training loss (default): 470.1152, validation loss: 116.0891
2025-05-20 00:50:11 [INFO]: Epoch 006 - training loss (default): 469.1584, validation loss: 113.6763
2025-05-20 00:50:12 [INFO]: Epoch 007 - training loss (default): 468.8160, validation loss: 111.7024
2025-05-20 00:50:13 [INFO]: Epoch 008 - training loss (default): 468.5682, validation loss: 109.9479
2025-05-20 00:50:15 [INFO]: Epoch 009 - training loss (default): 468.4283, validation loss: 108.2748
2025-05-20 00:50:16 [INFO]: Epoch 010 - training loss (default): 468.3826, validation loss:

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[W 2025-05-20 00:52:53,386] Trial 12 failed with parameters: {'lr': 0.0005489003043475935, 'epochs': 37, 'batch_size': 32, 'length_scale': 0.5911738963397587, 'beta': 0.8838948069479122} because of the following error: MlflowException("API request to http://localhost:5000/api/2.0/mlflow/runs/get failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/get?run_uuid=622e0a78641f42fdbfa5a25f0cd22ed4&run_id=622e0a78641f42fdbfa5a25f0cd22ed4 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1a967cfd60>: Failed to establish a new connection: [Errno 111] Connection refused'))").
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/urllib3/util/conn

MlflowException: API request to http://localhost:5000/api/2.0/mlflow/runs/get failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/get?run_uuid=886d8065455e497ab608c33e67318f7c&run_id=886d8065455e497ab608c33e67318f7c (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1a966c33d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import GPVAE
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_samples, n_timesteps, n_features = imputation.shape
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        reshaped = imputation[:, :, i].reshape(-1, 1)
        inversed = scalers[i].inverse_transform(reshaped)
        imputation_denorm[:, :, i] = inversed.reshape(n_samples, n_timesteps)
    
    return imputation_denorm



#Optional: Artificially mask. Mask 20% of the data (MIT part). Try masking 30% here 
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))



class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("GP_VAE_2")


# Optuna objective function
def objective(trial):
    params = {
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50),
        "batch_size": trial.suggest_int("batch_size", 32, 128, step=32),
        "length_scale": trial.suggest_float("length_scale",0.5,5.0),
        "beta": trial.suggest_float("beta",0.1,1.0)
 }

    with mlflow.start_run(run_name="GP-VAE-Trial", nested=True) as run:
        mlflow.log_params(params)

        gp_vae = GPVAE(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            latent_size=37, #should be the latent dimensions 
            encoder_sizes=(128,128), #should I change it here too?
            decoder_sizes=(256,256), #should I change the model size?
            kernel="cauchy",
            beta=params["beta"], #The weight of KL divergence in ELBO
            M=1,  #The number of Monte Carlo samples for ELBO estimation during training.
            K=1,  #The number of importance weights for IWAE model training loss.
            sigma=1.005, # The scale parameter for a kernel function
            length_scale=params["length_scale"], #The length scale parameter for a kernel function
            kernel_scales=1, #The number of different length scales over latent space dimensions
            window_size=24,  # Window size for the inference CNN.
            batch_size=params["batch_size"],
            # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
            epochs=params["epochs"],
            # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
            # You can leave it to defualt as None to disable early stopping.
            patience=3,
            # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
            # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
            optimizer=Adam(lr=params["lr"]),
            # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
            # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
            # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
            num_workers=0,
            # just leave it to default as None, PyPOTS will automatically assign the best device for you.
            # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
            device=device,
            # set the path for saving tensorboard and trained model files 
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model",
            # only save the best model after training finished.
            # You can also set it as "better" to save models performing better ever during training.
            model_saving_strategy="best",
        )




        # train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
        gp_vae.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        gp_vae_results = gp_vae.predict({"X": test_X_masked}, n_sampling_times=2)
        gp_vae_imputation = gp_vae_results["imputation"]

        print(f"The shape of gp_vae_imputation is {gp_vae_imputation.shape}")

        # for error calculation, we need to take the mean value of the multiple samplings for each data sample
        mean_gp_vae_imputation = gp_vae_imputation.mean(axis=1)

        test_imputation_denorm = inverse_scale(mean_gp_vae_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)


         # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse_i)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", avg_mae)
        mlflow.log_metric("avg_rmse", avg_rmse)

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("GP-VAE-2")
with mlflow.start_run(run_name="GPVAE_Optuna_Study(2)") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]


    
    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

    print("Best Parameters:", best_params)
    print("Best Objective Value:", best_value)

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[I 2025-05-20 00:58:15,405] Trial 0 finished with value: 7669.867760159568 and parameters: {'lr': 0.00025886707779089533, 'epochs': 47, 'batch_size': 96, 'length_scale': 2.9032511643432954, 'beta': 0.6220489199186152}. Best is trial 0 with value: 7669.867760159568.
2025-05-20 00:58:15 [INFO]: Using the given device: cuda
2025-05-20 00:58:15 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T005815
2025-05-20 00:58:15 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/gp_vae/best_model/20250520_T005815/tensorboard
2025-05-20 00:58:15 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 209,678


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/27e36dc1f0814678894742cae6a5e15c
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


...

🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/a87360c5d14a499d9e48c7ec1c8b18e2
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318


2025-05-20 01:07:10 [INFO]: Epoch 001 - training loss (default): 14176.3963, validation loss: 17276.3695
2025-05-20 01:07:11 [INFO]: Epoch 002 - training loss (default): 6433.6481, validation loss: 16789.6317
2025-05-20 01:07:12 [INFO]: Epoch 003 - training loss (default): 5533.9209, validation loss: 16617.4325
2025-05-20 01:07:12 [INFO]: Epoch 004 - training loss (default): 5482.6444, validation loss: 16533.7953
2025-05-20 01:07:13 [INFO]: Epoch 005 - training loss (default): 5467.5532, validation loss: 16467.5840
2025-05-20 01:07:14 [INFO]: Epoch 006 - training loss (default): 5459.5771, validation loss: 16410.7124
2025-05-20 01:07:14 [INFO]: Epoch 007 - training loss (default): 5454.8097, validation loss: 16362.3166
2025-05-20 01:07:15 [INFO]: Epoch 008 - training loss (default): 5451.8163, validation loss: 16318.2273
2025-05-20 01:07:16 [INFO]: Epoch 009 - training loss (default): 5449.7641, validation loss: 16278.8545
2025-05-20 01:07:16 [INFO]: Epoch 010 - training loss (default)

The shape of gp_vae_imputation is (1056, 2, 20, 31)


[I 2025-05-20 01:07:35,670] Trial 19 finished with value: 7742.206275543968 and parameters: {'lr': 0.00024272238891417084, 'epochs': 36, 'batch_size': 96, 'length_scale': 3.5478580789988197, 'beta': 0.40201733936255823}. Best is trial 4 with value: 7664.664779482548.


🏃 View run GP-VAE-Trial at: http://localhost:5000/#/experiments/832352739106302318/runs/be94307aeecf4034a1e4d0c9fd03f9e4
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
Best Parameters: {'lr': 0.00012021352690757769, 'epochs': 43, 'batch_size': 64, 'length_scale': 2.3487754392417886, 'beta': 0.45088192430534046}
Best Objective Value: 7664.664779482548
🏃 View run GPVAE_Optuna_Study(2) at: http://localhost:5000/#/experiments/832352739106302318/runs/fc110587ae72462385114c2ee013d857
🧪 View experiment at: http://localhost:5000/#/experiments/832352739106302318
